In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import json
from sklearn.feature_selection import VarianceThreshold

In [2]:
meta = pd.read_csv('../data/FFMetadata_v10.csv',low_memory=False)
df = pd.read_csv('../data/FFChallenge_v5/background.csv',low_memory=False,index_col=0)
constant = pd.read_csv('../data/FFChallenge_v5/constantVariables.txt',header=None,index_col=0)

print(f'Background: {df.shape[1]} variables')

print(f'Meta: {meta.shape[0]} variables')

print(f'Constant: {constant.shape[0]} variables')

Background: 13026 variables
Meta: 25684 variables
Constant: 2432 variables


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4242 entries, 1 to 4242
Columns: 13026 entries, cf1intmon to k5f1
dtypes: float64(544), int64(12366), object(116)
memory usage: 421.6+ MB


In [4]:
# df = df[df.columns[df.columns.isin(meta.new_name)]]

df.columns[df.columns.isin(meta.new_name)]

Index(['cf1intmon', 'cf1intyr', 'cf1lenhr', 'cf1lenmin', 'cf1twoc', 'cf1fint',
       'cf1natsm', 'f1natwt', 'cf1natsmx', 'f1natwtx',
       ...
       'q5citywt_rep63', 'q5citywt_rep64', 'q5citywt_rep65', 'q5citywt_rep66',
       'q5citywt_rep67', 'q5citywt_rep68', 'q5citywt_rep69', 'q5citywt_rep70',
       'q5citywt_rep71', 'q5citywt_rep72'],
      dtype='object', length=12840)

# Drop constantVariables.txt

In [5]:
fixed = df.drop(constant.index.values,axis=1)

print(f'Background without constant: {fixed.shape[1]} variables')

Background without constant: 10594 variables


# Drop columns with excessive real missing values

In [6]:
def dropvars(df,nan_threshold=0.8):
    # thresh = Require that many non-NA values 
    return df.dropna(thresh=len(df)*nan_threshold, axis=1)

filled = dropvars(fixed)

filled

,cf1lenhr,cf1lenmin,cf1fint,cf1citsm,f1citywt,f1a2,f1a3,f1a4,f1a4a,f1a5,...,m4d9,m4e23,f4d6,f4d7,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1
challengeID,,,,,,,,,,,,,,,,,,,,,
1,-9,-9,0,-9,-3.000000,-9,-9,-9,-9,-9,...,6.269946,5.180325,2.511131,1.718804,6.473537,16.369411,4.476881,9.628369,15.981275,24.038266
2,0,40,1,1,68.455658,2,1,1,-6,1,...,6.269946,27.680196,2.511131,1.718804,6.473537,16.369411,26.671897,9.628369,15.981275,3.667679
3,0,45,1,1,42.319057,1,1,1,-6,1,...,6.269946,5.180325,20.867881,24.115867,6.473537,16.369411,4.476881,9.628369,15.981275,24.038266
4,0,45,1,1,25.628830,1,1,1,-6,1,...,6.269946,5.180325,22.018875,22.932641,6.473537,-5.169243,4.476881,9.628369,-6.303171,4.140511
5,-6,50,1,1,41.954487,2,1,1,-6,1,...,6.269946,5.180325,22.916602,22.988036,6.473537,-6.034660,4.476881,9.628369,-6.211828,3.668879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,-9,-9,0,-9,-3.000000,-9,-9,-9,-9,-9,...,27.941956,5.180325,2.511131,1.718804,6.473537,16.369411,4.476881,9.628369,15.981275,24.038266
4239,0,40,1,1,64.195152,2,1,1,-6,1,...,28.703663,5.180325,2.511131,1.718804,6.473537,16.369411,4.476881,9.628369,15.981275,3.424922
4240,-9,-9,0,-9,-3.000000,-9,-9,-9,-9,-9,...,6.269946,26.990283,2.511131,1.718804,6.473537,16.369411,4.476881,9.628369,15.981275,24.038266


In [7]:
print(f'Background without excessive missing/low-variance values: {filled.shape[1]} variables')

Background without excessive missing/low-variance values: 10479 variables


# Select only variables with metadata

In [8]:
cols = [col for col in filled.columns if col in meta.new_name.values]

filled[cols]

,cf1lenhr,cf1lenmin,cf1fint,cf1citsm,f1citywt,f1a2,f1a3,f1a4,f1a4a,f1a5,...,cf5hhincb,cf5hhimpb,cm5povco,cf5povco,cf5povcob,cm5povca,cf5povca,cf5povcab,cm5samp,cf5samp
challengeID,,,,,,,,,,,,,,,,,,,,,
1,-9,-9,0,-9,-3.000000,-9,-9,-9,-9,-9,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,6,6
2,0,40,1,1,68.455658,2,1,1,-6,1,...,-9.000000,-9,0.842485,-9.000000,-9.000000,1,-9,-9,-7,1
3,0,45,1,1,42.319057,1,1,1,-6,1,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,7,7
4,0,45,1,1,25.628830,1,1,1,-6,1,...,55377.768078,1,2.755591,1.807249,2.722656,4,3,4,-7,-7
5,-6,50,1,1,41.954487,2,1,1,-6,1,...,8347.302339,1,0.744948,0.951748,0.970510,1,2,1,-7,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,-9,-9,0,-9,-3.000000,-9,-9,-9,-9,-9,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,3,3
4239,0,40,1,1,64.195152,2,1,1,-6,1,...,3668.270229,5,0.977176,0.424686,0.965887,2,2,2,-7,-7
4240,-9,-9,0,-9,-3.000000,-9,-9,-9,-9,-9,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,5,6


# Remove variables malformed

In [9]:
# dtypes[dtypes.new_name == 'cf4fint']
# string = '1960-01-02'
# mask = df.isin(['1960-01-01']).any()
# df.columns[mask]

malformed = {
    'r3b26_a': 'Has text instead of numerical encodings',
    'cf4fint': 'cf4fint encoded as binary but has 1960-01-01',
}

# remove malformed keys from cols if they exist
for key in malformed.keys():
    if key in cols:
        cols.remove(key)
        print(f'Removed {key} from cols')

Removed cf4fint from cols


# Explore variables
## By type of variable

In [10]:
meta.source.value_counts()

questionnaire    12619
restricted        8350
constructed       2206
weight            2176
saliva             263
actigraphy          69
idnum                1
Name: source, dtype: int64

In [11]:
meta[meta.new_name.isin(cols)].source.value_counts()

questionnaire    9746
constructed       523
weight            121
Name: source, dtype: int64

## By datatypes and topics

In [12]:
meta[meta.new_name.isin(cols)].type.value_counts()

Binary                   4800
Ordered Categorical      2987
Continuous               1628
Unordered Categorical     975
Name: type, dtype: int64

In [13]:
meta[meta.new_name.isin(cols)].topics.value_counts().head(15)

Finances                                   1633
Health and health behavior                 1071
Housing and neighborhood                    858
Parenting                                   833
Cognitive and behavioral development        730
Romantic relationships                      674
Education and school                        591
Childcare                                   504
Employment                                  307
Paradata and weights                        251
Housing and neighborhood ; Demographics     238
Legal system                                206
Family and social ties                      201
Demographics                                129
Attitudes and expectations                  108
Name: topics, dtype: int64

# Export metadata

In [14]:
myvars = meta[meta.new_name.isin(cols)].copy()

myvars['one_topic'] = myvars['topics'].str.split(';').str[0]

# trim one topic
myvars['one_topic'] = myvars['one_topic'].str.strip()

# remove space and upper case
myvars['one_topic'] = myvars['one_topic'].str.replace(' ','_').str.lower()

dtypes = myvars[['new_name','varlab','type','one_topic']].copy()

dtypes

,new_name,varlab,type,one_topic
3,cf1lenhr,What was the total length of interview - Hours,Continuous,paradata_and_weights
4,cf1lenmin,What was the total length of interview - Minutes,Continuous,paradata_and_weights
6,cf1fint,Constructed - Was father interviewed at baseline?,Binary,paradata_and_weights
11,cf1citsm,Constructed - Baseline city sample flag,Binary,paradata_and_weights
12,f1citywt,Father baseline city sample weight (20-cities ...,Continuous,paradata_and_weights
...,...,...,...,...
12790,cm5povca,Constructed - Mother's poverty category at 9-year,Ordered Categorical,finances
12791,cf5povca,Constructed - Father's poverty category at 9-year,Ordered Categorical,finances
12792,cf5povcab,Constructed - Father's poverty category at 9-y...,Ordered Categorical,finances
12793,cm5samp,Constructed - Y9 reason for mother non-response,Unordered Categorical,paradata_and_weights


In [15]:
ordcat_list = dtypes[dtypes.type == 'Ordered Categorical']

dictdtype = {
    'Binary': 'object', # binary encoded with 1/2 instead of 0/1 so we will convert it later
    'Ordered Categorical': CategoricalDtype(categories=ordcat_list, ordered=True),
    # 'Ordered Categorical to integer': 'int8',
    # 'Ordered Categorical': 'object', 
    'Continuous': 'float64',
    'Unordered Categorical': 'category',
}

dtypes['dtype'] = dtypes.type.map(dictdtype)

miscores = pd.read_csv('../metadata/miscores.csv',index_col=0)

dtypes = dtypes.set_index('new_name').join(miscores).reset_index()

dtypes

,new_name,varlab,type,one_topic,dtype,gpa,grit,materialHardship,eviction,layoff,jobTraining
0,cf1lenhr,What was the total length of interview - Hours,Continuous,paradata_and_weights,float64,0.000000,0.009892,0.003785,0.000000,0.007810,0.000000
1,cf1lenmin,What was the total length of interview - Minutes,Continuous,paradata_and_weights,float64,0.000000,0.000000,0.000000,0.001409,0.000000,0.000000
2,cf1fint,Constructed - Was father interviewed at baseline?,Binary,paradata_and_weights,object,0.021894,0.018859,0.016743,0.028699,0.000000,0.000000
3,cf1citsm,Constructed - Baseline city sample flag,Binary,paradata_and_weights,object,0.012485,0.023336,0.011364,0.000000,0.016590,0.004293
4,f1citywt,Father baseline city sample weight (20-cities ...,Continuous,paradata_and_weights,float64,0.000000,0.000000,0.000000,0.009234,0.002574,0.014284
...,...,...,...,...,...,...,...,...,...,...,...
10385,cm5povca,Constructed - Mother's poverty category at 9-year,Ordered Categorical,finances,category,0.000000,0.000000,0.033440,0.031431,0.000000,0.015174
10386,cf5povca,Constructed - Father's poverty category at 9-year,Ordered Categorical,finances,category,0.000000,0.030205,0.000000,0.000000,0.000000,0.001519
10387,cf5povcab,Constructed - Father's poverty category at 9-y...,Ordered Categorical,finances,category,0.000000,0.000000,0.000000,0.016226,0.000454,0.000983
10388,cm5samp,Constructed - Y9 reason for mother non-response,Unordered Categorical,paradata_and_weights,category,0.000000,0.015793,0.013264,0.000000,0.000000,0.000000


# Export metadata.json

In [16]:
dtypes_dict = dict(zip(dtypes.new_name,dtypes.dtype))
dtypes_dict = {'challengeID': 'int64', **dtypes_dict}

dtypes_dict = {k: str(v) if isinstance(v, CategoricalDtype) else v for k, v in dtypes_dict.items()}

with open('../metadata/metadata.json', 'w') as fp:
    json.dump(dtypes_dict, fp)

In [17]:
pd.read_csv('../data/FFChallenge_v5/background.csv',low_memory=False, \
                            dtype=dtypes_dict,\
                            usecols=dtypes_dict.keys()).set_index('challengeID')

,cf1lenhr,cf1lenmin,cf1fint,cf1citsm,f1citywt,f1a2,f1a3,f1a4,f1a4a,f1a5,...,cf5hhincb,cf5hhimpb,cm5povco,cf5povco,cf5povcob,cm5povca,cf5povca,cf5povcab,cm5samp,cf5samp
challengeID,,,,,,,,,,,,,,,,,,,,,
1,-9.0,-9.0,0,-9,-3.000000,-9,-9,-9,-9,-9,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,6,6
2,0.0,40.0,1,1,68.455658,2,1,1,-6,1,...,-9.000000,-9,0.842485,-9.000000,-9.000000,1,-9,-9,-7,1
3,0.0,45.0,1,1,42.319057,1,1,1,-6,1,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,7,7
4,0.0,45.0,1,1,25.628830,1,1,1,-6,1,...,55377.768078,1,2.755591,1.807249,2.722656,4,3,4,-7,-7
5,-6.0,50.0,1,1,41.954487,2,1,1,-6,1,...,8347.302339,1,0.744948,0.951748,0.970510,1,2,1,-7,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,-9.0,-9.0,0,-9,-3.000000,-9,-9,-9,-9,-9,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,3,3
4239,0.0,40.0,1,1,64.195152,2,1,1,-6,1,...,3668.270229,5,0.977176,0.424686,0.965887,2,2,2,-7,-7
4240,-9.0,-9.0,0,-9,-3.000000,-9,-9,-9,-9,-9,...,-9.000000,-9,-9.000000,-9.000000,-9.000000,-9,-9,-9,5,6


In [18]:
dtypes.to_csv('../metadata/metadata.csv',index=False)